In [1]:
import torch

In [7]:
print(torch.cuda.is_available())      # GPU 사용 가능 여부 (True/False)
print(torch.cuda.device_count())      # 사용 가능한 GPU 개수
print(torch.cuda.get_device_name(0)) 

True
1
NVIDIA GeForce RTX 5070


c:\Users\USER\.conda\envs\pytorch_test\lib\site-packages\torch\cuda\__init__.py:235: UserWarning: 
NVIDIA GeForce RTX 5070 with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 sm_90.
If you want to use the NVIDIA GeForce RTX 5070 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(



---

# 📌 PyTorch Device 정리

## 1. 기본 개념

* **device**: 텐서/모델이 어디(CPU/GPU/가속기)에 올라갈지 정하는 객체
* 형태: `torch.device(type, index)`

  * **type**: `"cpu"`, `"cuda"`, `"mps"`, `"xpu"`, `"rocm"`
  * **index**: 같은 타입 장치가 여러 개일 때 번호 (`cuda:0`, `cuda:1` …)

---

## 2. 주요 device 유형

| 유형       | 대상 하드웨어           | 비고              |
| -------- | ----------------- | --------------- |
| **cpu**  | CPU               | 항상 사용 가능        |
| **cuda** | NVIDIA GPU        | 가장 많이 쓰임        |
| **mps**  | Apple Silicon GPU | 맥북 M1/M2용       |
| **xpu**  | Intel GPU         | Intel oneAPI 기반 |
| **rocm** | AMD GPU           | AMD GPU 지원      |

---

## 3. device 인덱스 (index)

* GPU가 여러 개일 경우:

  * `cuda:0` → 0번 GPU
  * `cuda:1` → 1번 GPU …
* 확인법:

  ```python
    print("count:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        props = torch.cuda.get_device_properties(i)
        print(i, props.name, props.total_memory // (1024**2), "MB")
  ```

---

## 4. 난수 생성 시에도 device 지정하는 이유

* `torch.randn((2,3), device="cuda:0")`
  → 텐서가 바로 GPU에 생성됨
* 만약 CPU에서 만든 뒤 `.to("cuda:0")` 하면 **CPU 메모리 + GPU 복사 오버헤드** 발생
* 즉, **메모리/성능 최적화를 위해 처음부터 device를 지정하는 게 가장 효율적**

---

## 5. `torch.device(n)` 정수만 쓰는 경우

* `torch.device(0)` → 현재 기본 가속기가 **cuda**라면 `cuda:0`으로 해석됨
* `torch.device(1)` → 기본 가속기가 **xpu**라면 `xpu:1`이 될 수 있음
* 가속기가 없으면 **RuntimeError 발생**
* **주의**: 마지막에 만든 device와는 무관함

---

## 6. CUDA_VISIBLE_DEVICES

* 환경변수: **어떤 GPU를 보여줄지, 어떤 순서로 보여줄지** 제한/재매핑
* 예:

  ```bash
  export CUDA_VISIBLE_DEVICES=2,3
  ```

  → PyTorch 입장에서

  * `cuda:0` = 실제 물리 GPU 2
  * `cuda:1` = 실제 물리 GPU 3
* 목적: **서버에서 여러 사용자가 동시에 쓸 때 충돌 방지**

---

## ✅ 최종 요약

* **device = 텐서/모델이 올라갈 위치**
* 자주 쓰는 건 `"cpu"`, `"cuda:i"`
* 학습할 땐 난수/텐서를 바로 GPU에 생성해야 메모리 효율적
* `torch.device(n)`은 기본 accelerator 기준으로만 동작 → 안전하게 쓰려면 항상 `"cuda:0"` 식으로 명시
* 서버 환경에서는 `CUDA_VISIBLE_DEVICES`로 GPU 충돌을 방지하고 관리

---

👉 현태님, 원하면 제가 이 내용을 **그림(흐름도)**처럼 정리해서 “로컬 PC (GPU 1개)” vs “서버 (GPU 여러 개)” 상황을 비교해드릴까요?




In [ ]:
# torch.device() : device 객체를 만든다
# 실제로 device가 없는 환경에서도 이 객체는 만들어진다.
# 즉, CUDA GPU가 없으면 바로 에러는 안 나고, 그 텐서를 실제로 만들 때 에러 발생

print(torch.device('cuda:0'))


print(torch.device('cpu'))


print(torch.device('mps'))


print(torch.device('cuda'))  # implicit index is the "current device index"


cuda:0
cpu
mps
cuda


In [ ]:
# 장치 유형 및 장치 순서 

# NVIDIA GPU 0번을 가리키는 device 객체 생성
torch.device('cuda', 0)

# Apple Silicon(M1/M2) GPU를 가리키는 device 객체 생성
torch.device('mps', 0)

# CPU를 가리키는 device 객체 
# CPU는 인덱스 의미가 없어서 0을 생략해도됨
torch.device('cpu', 0)

False

In [ ]:
# with torch.device :  컨텍스트 시작 → 기본 device를 cuda:0로 설정
# r = torch.randn(2,3) : tensor r을 생성하면 자동으로 cuda:0 에 올라감
# r.device : r이 실제로 있는 장치 확인

with torch.device('cuda:0'):       
    r = torch.randn(2, 3)         
r.device

device(type='cpu')

In [ ]:
# cuda0은 torch.device의 객체
# device= 인자로 객체로 전달하면 해당 장치에 바로 텐서가 생성된다.
cuda0 = torch.device('cuda:0')
torch.randn((2,3), device=cuda0)

# 문자열로 대체 가능
x = torch.randn((2,3), device='cuda:0')
print(x.device)   # cuda:0

In [ ]:

# 프로젝트 전체에서 device를 한 번만 지정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 이후 모든 텐서/모델에서 공통적으로 사용
x = torch.randn((2,3), device=device)
model.to(device)


---

# 📌 PyTorch에서 스칼라 자동 전송 정리

* **일반 규칙**: CPU ↔ GPU 간 텐서는 **자동 이동 안 됨**, 반드시 `.to(device)` 필요
* **예외**: **스칼라 텐서 (dim()==0)** 는 CPU → GPU로 연산 시 자동 전송됨
* **GPU → CPU 자동 전송은 없음**
* **실무 활용**: 손실값(loss), 작은 상수(예: `+ 1.0`, `* 0.5`) 같은 경우에만 편의상 쓰일 뿐, 대부분은 `.to(device)`로 맞추는 게 안전

---

👉 한 줄 요약:
**PyTorch는 스칼라만 CPU→GPU 자동 전송 허용, 나머지는 전부 수동으로 `.to(device)` 해야 한다.**


In [ ]:
import torch

# 1) CPU 스칼라 + GPU 벡터 (자동 전송 O)
a = torch.ones(())            # CPU scalar
b = torch.ones(3).cuda()      # GPU vector
print((a + b).device)         # cuda:0

# 2) GPU 스칼라 + CPU 벡터 (자동 전송 X → 에러)
try:
    a = torch.ones(()).cuda()   # GPU scalar
    b = torch.ones(3)           # CPU vector
    print((a + b).device)
except Exception as e:
    print("Error:", e)
